In [1]:
from cache_analysis.cache.frontend import CacheConfig, in_out_nodes_generation, inst_block_gen, extract_mem_blocks
from cache_analysis.cache.fixpoint import Fixpoint, FixpointType, fixpoint_nodes_generation
from pprint import pprint

In [ ]:
"""
    从CFG到cache analysis的接口 需要输入五个信息
    有关CFG的信息为:
        1. 所有 basic block 的标识 相当于其identity 必须是唯一的 这里直接用我们之前使用的 basic block的首指令地址, 相当于blockHead
        2. 入口 basic block , 也就是整个CFG的头, 这里可以理解为main函数的开始 相当于blockHead[1]
        3. 边信息 这里是有向边 从左指向右 ---> 构成一个元组 有几条边就有几个元组 所有元组构成一个总的边列表
    有关cache配置的信息：
        1. 只要知道地址中的偏移位和set_index位以及cache是几路相联即可 or 输入直接得到的信息也可以 比如cache总大小, cache line 的大小, 以及几路相联 也能根据关系计算出来 cache line size ---> offset_len;
    有关访存的信息：
        1. 需要输入每个 basic block 所访存的地址信息, 比如指令的则是 开始 - 结尾, dict类型, basic block identity ---> key ; tuple(start,end) ---> value
"""
example = 1
if example == 0:
    # Idents of all nodes.
    all_nodes = ['400604', '400630', '40066c', '400708', '400724', '400730', '40073c', '400740', '40074c', '400754', '400758', '400770', '40079c', '4007c0', '4007d4', '4007fc', '40080c', '400848', '400864', '400870', '400874']
    entrynode = '400604'
    # All directed edges. The CFG is determined by nodes and edges.
    edges = [('400604', '400740'), ('400604', '400630'), ('400630', '400730'), ('40066c', '400724'), ('40066c', '400708'), ('400708', '400874'), ('400724', '400730'), ('400730', '40066c'), ('400730', '40073c'), ('40073c', '400870'), ('400740', '400754'), ('400740', '40074c'), ('40074c', '400758'), ('400754', '400758'), ('400758', '400864'), ('400770', '4007c0'), ('400770', '40079c'), ('40079c', '400848'), ('4007c0', '4007fc'), ('4007d4', '4007fc'), ('4007fc', '4007d4'), ('4007fc', '40080c'), ('40080c', '400848'), ('400848', '400864'), ('400864', '400770'), ('400864', '400870'), ('400870', '400874')]
    # For instruction cache analysis, the instruction range of each basic block.
    address_range = {'400604': (4195844, 4195884), '400630': (4195888, 4195944), '40066c': (4195948, 4196100), '400708': (4196104, 4196128), '400724': (4196132, 4196140), '400730': (4196144, 4196152), '40073c': (4196156, 4196156), '400740': (4196160, 4196168), '40074c': (4196172, 4196176), '400754': (4196180, 4196180), '400758': (4196184, 4196204), '400770': (4196208, 4196248), '40079c': (4196252, 4196284), '4007c0': (4196288, 4196304), '4007d4': (4196308, 4196344), '4007fc': (4196348, 4196360), '40080c': (4196364, 4196420), '400848': (4196424, 4196448), '400864': (4196452, 4196460), '400870': (4196464, 4196464), '400874': (4196468, 4196472)}

    # Cache config.
    cache_conf = CacheConfig(offset_len=6, set_index_len=8, assoc=4)

else:
    # Idents of all nodes.
    all_nodes = ['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', 'n15', 'n16']
    entrynode = 'n0'
    # All directed edges. The CFG is determined by nodes and edges.
    edges = [('n0', 'n3'), ('n13', 'n1'), ('n1', 'n14'), ('n14', 'n2'), ('n3', 'n12'), ('n4', 'n10'), ('n5', 'n8'), ('n6', 'n15'), ('n15', 'n7'), ('n7', 'n8'), ('n8', 'n6'), ('n8', 'n9'), ('n9', 'n10'), ('n10', 'n5'), ('n10', 'n11'), ('n11', 'n12'), ('n12', 'n4'), ('n12', 'n13')]
    # For instruction cache analysis, the instruction range of each basic block.
    address_range = {'n0': ((1916, 1920), (1920, 1924), (1924, 1928), (1928, 1932), (1932, 1936), (1936, 1940), (69624, 69628), (1940, 1944), (1944, 1948), (65536, 65540), (1948, 1952), (1952, 1956), (69624, 69628), (1956, 1960), (65544, 65548), (1960, 1964), (1964, 1968), (69624, 69628), (1968, 1972)), 'n1': ((1972, 1976), (1976, 1980), (69624, 69628), (1980, 1984), (1984, 1988), (89976, 89980), (1988, 1992), (1992, 1996), (1996, 2000)), 'n2': ((2000, 2004), (2004, 2008), (2008, 2012)), 'n3': ((2012, 2016), (2016, 2020), (2020, 2024), (2024, 2028), (2028, 2032)), 'n4': ((2032, 2036), (2036, 2040)), 'n5': ((2040, 2044), (2044, 2048)), 'n6': ((2048, 2052), (2052, 2056), (2056, 2060), (2060, 2064), (2064, 2068)), 'n7': ((2068, 2072), (2072, 2076), (2076, 2080)), 'n8': ((2080, 2084), (2084, 2088), (2088, 2092)), 'n9': ((2092, 2096), (2096, 2100), (2100, 2104)), 'n10': ((2104, 2108), (2108, 2112), (2112, 2116)), 'n11': ((2116, 2120), (2120, 2124), (2124, 2128)), 'n12': ((2128, 2132), (2132, 2136), (2136, 2140)), 'n13': ((2140, 2144), (2144, 2148), (2148, 2152), (2152, 2156)), 'n14': ((1632, 1636), (1636, 1640), (69552, 69556), (1640, 1644), (1644, 1648)), 'n15': ((2220, 2224), (2224, 2228), (2228, 2232), (2232, 2236), (2236, 2240), (2240, 2244), (2244, 2248), (2248, 2252), (2252, 2256), (2256, 2260), (2260, 2264), (2264, 2268), (2268, 2272), (2272, 2276), (2276, 2280), (2280, 2284), (2284, 2288), (2288, 2292), (2292, 2296), (2296, 2300), (2300, 2304), (2304, 2308), (2308, 2312), (2312, 2316), (2316, 2320), (65536, 65540), (2320, 2324), (2324, 2328), (2328, 2332)), 'n16': ((2332, 2336),)}

    # Cache config.
    cache_conf = CacheConfig(offset_len=6, set_index_len=8, assoc=4)

In [3]:
""" 获取每个 basic block 的 in_nodes 和 out_nodes (如果没有入 没有出 则标记位set() ) """

in_nodes, out_nodes = in_out_nodes_generation(all_nodes, edges)

pprint(in_nodes)
pprint(out_nodes)

{'n0': set(),
 'n1': {'n13'},
 'n10': {'n4', 'n9'},
 'n11': {'n10'},
 'n12': {'n11', 'n3'},
 'n13': {'n12'},
 'n14': {'n1'},
 'n15': {'n6'},
 'n16': set(),
 'n2': {'n14'},
 'n3': {'n0'},
 'n4': {'n12'},
 'n5': {'n10'},
 'n6': {'n8'},
 'n7': {'n15'},
 'n8': {'n5', 'n7'},
 'n9': {'n8'}}
{'n0': {'n3'},
 'n1': {'n14'},
 'n10': {'n5', 'n11'},
 'n11': {'n12'},
 'n12': {'n13', 'n4'},
 'n13': {'n1'},
 'n14': {'n2'},
 'n15': {'n7'},
 'n16': set(),
 'n2': set(),
 'n3': {'n12'},
 'n4': {'n10'},
 'n5': {'n8'},
 'n6': {'n15'},
 'n7': {'n8'},
 'n8': {'n6', 'n9'},
 'n9': {'n10'}}


In [4]:
""" basic block 到 memory block 的映射; 每个 basic block 访问了 那些 memory block; 且 memory block 由 tag + set_index 唯一确定 """

mem_block_dict = {ident: inst_block_gen(*address_range[ident], cache_config=cache_conf) for ident in all_nodes}
pprint(mem_block_dict)

TypeError: inst_block_gen() got multiple values for argument 'cache_config'

In [5]:
""" 按 set_index 分割内存块 只打印访问此 memory block 的信息 """

pprint(extract_mem_blocks(30, mem_block_dict))

NameError: name 'mem_block_dict' is not defined

In [ ]:
""" Fixpoint nodes basic generation. """
""" Illustrated with must analysis, set_index=31 as example. """
""" must分析例子, set_index=31 作为例子 也就是输出第31个set的must分析 这里在进行可视化时需要得到256个set的must分析 may分析 """

fp_nodes = fixpoint_nodes_generation(nodes=all_nodes, in_nodes=in_nodes, out_nodes=out_nodes,
                                     cache_config=cache_conf, evicted=False,
                                     mem_block_dict=extract_mem_blocks(31, mem_block_dict))
pprint(fp_nodes)

In [ ]:
''' 上面是初始化 下面进行的是具体分析 注意这里需要entrynode信息 修改FixpointType即可 分别为 Must May Persistent 目前Persistent的输出未知, 继续测试即可; 这里的输出 e.g., 256为 memory block 的tag号, 因为已经指定了set_index 所以只需要tag即可 '''
fixpoint = Fixpoint(ty=FixpointType.Must, entry=entrynode, all_nodes=fp_nodes)
print(fixpoint.pse_topsort_seq)

fixpoint.run()
pprint(fp_nodes)